# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [0]:
 # Basic imports.
import os
import time
import numpy as np
import torch

from torch import nn
from torch import optim
from torch.nn import functional as F

from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import models
from torchvision import datasets
from torchvision import transforms

from skimage import io

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [0]:
# Setting predefined arguments.
args = {
    'epoch_num': 50,      # Number of epochs.
    'lr': 5e-4,           # Learning rate.
    'weight_decay': 1e-5, # L2 penalty.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 100,    # Mini-batch size.
    'print_freq': 1,      # Printing frequency.
    'lambda_var': 1.0,    # Variational multiplier in loss.
    'num_gauss': 20,      # Number of gaussians in bottleneck of VAE.
    'num_samples': 8,     # Number of samples to be generated in evaluation.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

# Carregando o  MNIST

In [0]:
# Root directory for the dataset (to be downloaded).
root = './'

# Transformations over the dataset.
data_transforms = transforms.Compose([
    transforms.ToTensor()
])

# Setting datasets and dataloaders.
train_set = datasets.MNIST(root,
                           train=True,
                           download=True,
                           transform=data_transforms)
test_set = datasets.MNIST(root,
                          train=False,
                          download=False,
                          transform=data_transforms)

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          args['batch_size'],
                          num_workers=args['num_workers'],
                          shuffle=True)
test_loader = DataLoader(test_set,
                         args['batch_size'],
                         num_workers=args['num_workers'],
                         shuffle=False)

# Printing training and testing dataset sizes.
print('Size of training set: ' + str(len(train_set)) + ' samples')
print('Size of test set: ' + str(len(test_set)) + ' samples')

# AutoEncoder Variacional

Idealmente codificações compactas de dados redundantes (i.e. imagens) deveriam produzir representações latentes que fossem independentes uma da outra num nível semântico. Ou seja, cada bin num feature map latente $z$ de um autoencoder deveria codificar o máximo de informação possível (i.e. linhas verticais que compõem um '1', '7' ou '9'; ou círculos que compõem um '6', '8' ou '0') para a reconstrução dos dígitos do MNIST, por exemplo. A [inferência variacional](https://www.cs.princeton.edu/courses/archive/fall11/cos597C/lectures/variational-inference-i.pdf) provém uma forma mais simples de computarmos o Maximum a Posteriori (MAP) de distribuições estatísticas complexas como as que estamos lidando.

![VAE Features](https://www.dropbox.com/s/fkvdn69tkh7tm1p/vae_gaussian.png?dl=1)

Se tivermos controle sobre representações latentes em $z$ que codificam features de algo nível semântico, podemos utilizar o Decoder de um AE para geração de novas amostras. Usando o Encoder de um AE tradicional, conseguimos partir do vetor de entrada $x$ e chegar no vetor latente $z \sim q(z ∣ x)$. Porém, como não temos controle sobre a distribuição $q$, não é possível fazer o caminho inverso, ou seja, a partir de $z$ modelar $x \sim p(x | z)$. Essa é a motivação para um Variational AutoEncoder (VAE).

![VAE x->z](https://www.dropbox.com/s/o8daaskdrhfav7r/VAE_Enc.png?dl=1)

![VAE z->x](https://www.dropbox.com/s/wqi8nsak84i11mi/VAE_Dec.png?dl=1)

Para podermos ter um controle maior sobre distribuição de cada bin de $z$, adicionamos uma "regularização" $\mathcal{L}_{KL}(\mu, \sigma)$ à loss de regressão $\mathcal{L}_{r}(x, \hat{x})$ de um AE tradicional. Percebe-se que $\mu$ e $\sigma$ devem codificar a média e o desvio padrão de distribuições gaussianas multivariadas, o que permite realizarmos uma amostragem dessa distribuição. Não podemos, porém, backpropagar de nós na nossa rede que realizem amostragem de uma distribuição. Portanto, precisamos do truque da reparametrização mostrado abaixo para backpropagarmos apenas por $\mu$ e $\sigma$, mas não por $\epsilon$.

![Reparametrization](https://jaan.io/images/reparametrization.png)

Assim, a arquitetura final de um VAE segue o esquema a seguir composto no bottleneck por um vetor $\mu$, um vetor $\sigma$ e um vetor $\epsilon$, que formam a representação latente $z = \mu + \sigma * \epsilon$.

![VAE training](https://www.dropbox.com/s/719vkfnfsobimmd/VAE_training.png?dl=1)

A ideia é que cada gaussiana codifique uma característica de alto nível nos dados, permitindo que utilizemos o modelo generativo do VAE para, de fato, gerar amostras novas verossímeis no domínio dos dados de treino.

![VAE gif](https://media.giphy.com/media/26ufgj5LH3YKO1Zlu/giphy.gif)

Para entender mais sobre "disentangled representations", ler o paper original do [VAE](https://arxiv.org/abs/1312.6114), o [$\beta$-VAE](https://openreview.net/references/pdf?id=Sy2fzU9gl) e o paper que propõe as [InfoGANs](https://arxiv.org/pdf/1606.03657.pdf):

# Atividade Prática: Implementando o VAE

1.   Defina a arquitetura do VAE. O Encoder da rede será composto de duas camadas precedendo as camadas $\mu$ e $\sigma$, de forma que $\mu$ e $\sigma$ recebam as mesmas entradas e se combinem como explicado a cima para formar o vetor latente $z$. A dimensionalidade de entrada dos dados ($784$) deve ser diminuída gradativamente até chegar no bottleneck, assim como nosso primeiro exemplo do AE Linear. Ambas camadas $\mu$ e $\sigma$ devem receber dados de dimensionalidade alta e codificá-los para uma saída dada pela variável *n_gaus*. Não é preciso criar uma camada explícita para $\epsilon$, já que ele só representa a amostragem de uma distribuição gaussiana ([torch.randn()](https://pytorch.org/docs/stable/torch.html#torch.randn));
2.   Complete a implementação dos métodos *encode()* que encapsula o forward pelo Encoder, *reparameterize()* que amostra $\epsilon$ e realiza o truque da reparametrização e *decode()* que faz o forward de $z$ pelo Decoder da rede, o qual deve ser simétrico ao encoder, ou seja, receber *n_gaus* features vindos de $z$ e gradativamente aumentar os features para recuperar $784$ features. Dica: na função *reparameterize()*;
3.   Defina a loss composta do VAE na função *variational_loss()*. Essa função deve retornar o componente $\mathcal{L}_{r}$ da loss (já feito usando a BCE) e o componente $\mathcal{L}_{KL}$. Dica: ver o Apêndice B do paper dos [VAEs](https://arxiv.org/pdf/1312.6114.pdf) para a fórmula de $\mathcal{L}_{KL}$;
4.   Na função *generate_2d()* altere as dimensões da tupla *dim_linspace* até achar um par de dimensões que influencie os novos samples em alto nível semântico.

# Definindo a arquitetura

In [0]:
# AutoEncoder implementation.
class VariationalAutoEncoder(nn.Module):
    
    def __init__(self, n_gaus):

        super(VariationalAutoEncoder, self).__init__()
        
        self.n_gaus = n_gaus
        
        # TO DO: Encoder.
        self.enc_1 = # ...
        self.enc_2 = # ...
        
        # TO DO: Layers mu and sigma.
        self.enc_mu = # ...
        self.enc_sigma = # ...
        
        # TO DO: Decoder.
        self.decoder = # ...
        
        self.initialize_weights()
        
    # TO DO: Encoding function.
    def encode(self, x):
        # ...
        
    # TO DO: Decoding function.
    def decode(self, z):
        # ...
               
    # TO DO: Reparametrization function. 
    def reparameterize(self, mu, logvar):
        
        std = torch.exp(0.5 * logvar)
        
        # TO DO: sample eps from gaussian.
        eps = # ...
        
        # TO DO: compute z using mu, eps and std.
        z = # ...
        
        # TO DO: return z.
    
    # Function for randomly initializing weights.
    def initialize_weights(self):
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        
        # TO DO: Call method encode().
        mu, logvar = # ...
        
        # TO DO: Use mu and logvar to compute z in method reparameterize().
        z = # ...
        
        # TO DO: Call method decode().
        dec = #...
        
        return dec, mu, logvar

# Instantiating architecture.
net = VariationalAutoEncoder(args['num_gauss']).to(args['device'])

# Printing architecture.
print(net)

# Definindo o otimizador

In [0]:
optimizer = optim.Adam(net.parameters(),
                       lr=args['lr'],
                       weight_decay=args['weight_decay'])

# Definindo a loss

In [0]:
# TO DO: Reconstruction + KL losses summed over all elements and batch.
def variational_loss(recon_x, x, mu, logvar):
    
    # Reconstruction loss using BCE.
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # TO DO: KL Divergence loss.
    # See Appendix B from VAE paper:
    #     https://arxiv.org/pdf/1312.6114.pdf.
    # See Pytorch's implementation of VAEs:
    #     https://github.com/pytorch/examples/blob/master/vae/main.py.
    KLD = # ...
    
    # TO DO: return BCE and KLD.

# Criando funções para Treino e Teste

In [0]:
# Training procedure.
def train(train_loader, net, optimizer, epoch):

    tic = time.time()
    
    # Setting network for training mode.
    net.train()

    # Lists for losses and metrics.
    train_loss = []
    
    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images and labels for batch.
        inps, labs = batch_data
        
        # Casting to cuda variables and reshaping.
        inps = inps.view(inps.size(0), -1).to(args['device'])
        
        # Clears the gradients of optimizer.
        optimizer.zero_grad()

        # Forwarding.
        outs, mu, logvar = net(inps)

        # TO DO Computing total loss.
        loss_bce, loss_kld = variational_loss(outs, inps, mu, logvar)
        loss = # ...

        # Computing backpropagation.
        loss.backward()
        optimizer.step()
        
        # Updating lists.
        train_loss.append((loss_bce.data.item(),
                           args['lambda_var'] * loss_kld.data.item(),
                           loss.data.item()))
    
    toc = time.time()
    
    train_loss = np.asarray(train_loss)
    
    # Printing training epoch loss and metrics.
    print('-------------------------------------------------------------------')
    print('[epoch %d], [train bce loss %.4f +/- %.4f], [train kld loss %.4f +/- %.4f], [training time %.2f]' % (
        epoch, train_loss[:,0].mean(), train_loss[:,0].std(), train_loss[:,1].mean(), train_loss[:,1].std(), (toc - tic)))
    print('-------------------------------------------------------------------')
    

In [0]:
# Testing procedure.
def test(test_loader, net, epoch):

    tic = time.time()
    
    # Setting network for evaluation mode.
    net.eval()

    # Lists for losses and metrics.
    test_loss = []
    
    # Iterating over batches.
    for i, batch_data in enumerate(test_loader):

        # Obtaining images and labels for batch.
        inps, labs = batch_data

        # Casting to cuda variables and reshaping.
        inps = inps.view(inps.size(0), -1).to(args['device'])

        # Forwarding.
        outs, mu, logvar = net(inps)

        # Computing loss.
        loss_bce, loss_kld = variational_loss(outs, inps, mu, logvar)
        loss = # ...
        
        # Updating lists.
        test_loss.append((loss_bce.data.item(),
                          args['lambda_var'] * loss_kld.data.item(),
                          loss.data.item()))
        
        if i == 0 and epoch % args['print_freq'] == 0:
            
            fig, ax = plt.subplots(2, 8, figsize=(16, 4))
        
        if i < 8 and epoch % args['print_freq'] == 0:
            
            ax[0, i].imshow(inps.view(inps.size(0), 28, 28)[0].detach().cpu().numpy())
            ax[0, i].set_yticks([])
            ax[0, i].set_xticks([])
            ax[0, i].set_title('Image ' + str(i + 1))
            
            ax[1, i].imshow(outs.view(inps.size(0), 28, 28)[0].detach().cpu().numpy())
            ax[1, i].set_yticks([])
            ax[1, i].set_xticks([])
            ax[1, i].set_title('Reconstructed ' + str(i + 1))
            
        if i == 8 and epoch % args['print_freq'] == 0:
            
            plt.show()
    
    toc = time.time()
    
    test_loss = np.asarray(test_loss)
    
    # Printing training epoch loss and metrics.
    print('-------------------------------------------------------------------')
    print('[epoch %d], [test bce loss %.4f +/- %.4f], [test kld loss %.4f +/- %.4f], [testing time %.2f]' % (
        epoch, test_loss[:,0].mean(), test_loss[:,0].std(), test_loss[:,1].mean(), test_loss[:,1].std(), (toc - tic)))
    print('-------------------------------------------------------------------')

In [0]:
# Evaluation procedure for sample generation.
def evaluate(net, n_samples, n_gauss):

    # Setting network for evaluation mode.
    net.eval()
    
    # Plotting new samples generated from VAE.
    fig, ax = plt.subplots(1, n_samples, figsize=(n_samples*2, 2))

    # Iterating over batches.
    for i in range(n_samples):
        
        # Sampling from Gaussian.
        sample = torch.randn(1, n_gauss).to(args['device'])
        
        # Forwarding through Decoder.
        sample = net.decode(sample).detach().cpu().view(28, 28).numpy()
        
        ax[i].imshow(sample)
        ax[i].set_yticks([])
        ax[i].set_xticks([])
        ax[i].set_title('New Sample ' + str(i + 1))
        
    plt.show()

# Iterando sobre epochs

In [0]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train(train_loader, net, optimizer, epoch)

    # Computing test loss and metrics.
    test(test_loader, net, epoch)
    
    # Evaluating sample generation in VAE.
    evaluate(net, args['num_samples'], args['num_gauss'])
    
    print('-- End of Epoch ---------------------------------------------------')
    print('-------------------------------------------------------------------')

In [0]:
# Evaluation procedure for sample generation.
def generate_2d(net, n_samples, n_gauss):

    # Setting network for evaluation mode.
    net.eval()
    
    # Creating linear space to visualize bivariate gaussian.
    linspace_gauss = torch.linspace(-2.5, 2.5, n_samples)
    
    # Select Gaussian dimensions
    dim_linspace = (0, 1)
    
    # Plotting.
    fig, ax = plt.subplots(n_samples, n_samples, figsize=(20, 20))

    for i in range(n_samples):
        
        for j in range(n_samples):

            # Filling batch with size 1 and n_gauss zeros of dimension.
            sample = torch.zeros(1, n_gauss).to(args['device'])
            
            # Replacing zeros in dimensions dim_linspace with values from
            # variable linspace_gauss.
            sample[0, dim_linspace[0]] = linspace_gauss[j]
            sample[0, dim_linspace[1]] = linspace_gauss[i]

            # Forwarding through decoder.
            sample = net.decode(sample).detach().cpu().view(28, 28).numpy()

            # Printing sample.
            ax[j, i].imshow(sample)
            ax[j, i].set_yticks([])
            ax[j, i].set_xticks([])
            ax[j, i].set_title('New Sample [' + str(j + 1) + ',' + str(i + 1) + ']')
        
    plt.show()
    
    
generate_2d(net, args['num_samples'], args['num_gauss'])